In [29]:
import tensorflow as tf

Load our sample data as one-hot vectors

In [30]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Start an interactive session, not sure why it's interactive

In [31]:
sess = tf.InteractiveSession()

Our input and output placeholders. The shapes start with `None` so we can batch multiple inputs

In [32]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

Make one layer of weights, plus bias

In [33]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

Our basic model is $\vec{x}\mathbf{W} + b = \vec{y}$

In [34]:
y = tf.matmul(x, W) + b

Optimize for cross-entropy

In [35]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
)

At each training interval, do one gradient descent step to minimize cross entropy

In [36]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

Must initialize our session

In [37]:
tf.global_variables_initializer().run()

Our MNINST data type has a "next batch" that will deliver, say, 100 records at a time

In [38]:
for _ in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [39]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [40]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [41]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9173


# Convolutional Neural Network

In [42]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [43]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [44]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [45]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [46]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [47]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [48]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [49]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [50]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [52]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(200):
        batch = mnist.train.next_batch(50)
        if i % 1000 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
          x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.08
test accuracy 0.911


In [58]:
sess = tf.Session()
type(sess)
sess.run(tf.global_variables_initializer())

In [25]:
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 1000 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

FailedPreconditionError: Attempting to use uninitialized value Variable_2
	 [[Node: Variable_2/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_2"], _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_2)]]

Caused by op 'Variable_2/read', defined at:
  File "/Users/jrpl/anaconda3/envs/libs2/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/jrpl/anaconda3/envs/libs2/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/jrpl/anaconda3/envs/libs2/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/jrpl/anaconda3/envs/libs2/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/jrpl/anaconda3/envs/libs2/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/jrpl/anaconda3/envs/libs2/lib/python3.5/site

In [60]:
mninst = input_data.read_data_sets('MNINST_data', one_hot=True)
batch = mninst.train.next_batch(50)

Extracting MNINST_data/train-images-idx3-ubyte.gz
Extracting MNINST_data/train-labels-idx1-ubyte.gz
Extracting MNINST_data/t10k-images-idx3-ubyte.gz
Extracting MNINST_data/t10k-labels-idx1-ubyte.gz


In [62]:
x

<tf.Tensor 'Placeholder_3:0' shape=(?, 784) dtype=float32>

In [63]:
y_

<tf.Tensor 'Placeholder_4:0' shape=(?, 10) dtype=float32>

In [64]:
keep_prob

<tf.Tensor 'Placeholder_5:0' shape=<unknown> dtype=float32>

In [61]:
train_accuracy = accuracy.eval(
    feed_dict={
        x: batch[0],
        y_: batch[1],
        keep_prob: 1.0
    }
)

FailedPreconditionError: Attempting to use uninitialized value Variable_19
	 [[Node: Variable_19/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_19"], _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_19)]]

Caused by op 'Variable_19/read', defined at:
  File "/Users/jrpl/anaconda3/envs/libs2/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/jrpl/anaconda3/envs/libs2/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/jrpl/anaconda3/envs/libs2/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/jrpl/anaconda3/envs/libs2/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/jrpl/anaconda3/envs/libs2/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/jrpl/anaconda3/envs/libs2/lib/python3.5